# Сравнение распределений в R

## 0\. Подготовительные работы

In [ ]:
getwd()

In [ ]:
data <- read.csv("data_compare_distr.csv")

In [ ]:
library(jsonlite)
library(dplyr)

In [ ]:
dim(data)

In [ ]:
str(data)

## 1\. Exploratory Analysis